In [1]:
import pandas as pd
from statistics import mean, median
import sys
import pprint
import pickle

In [2]:
project_ytest_lib = {}

In [3]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [4]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [5]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [6]:
def separate_versions(results):
    version_dfs = []
    
    for i in range(1,11):
        ver = results[ results['version']==i]
        version_dfs.append(ver)
    return version_dfs

In [7]:
def get_sbs_delays(ci, y_test, batch_size):
    
    
    y_test = output_values(y_test)
    
    sbs_list = []
    missed = []
    b = batch_size
    
    i = 0

    while i < len(ci):

        if ci[i] == 0:
            if y_test[i] == 0:
                sbs_list.append(0)

            while len(missed) > 0:
                ind = missed.pop()
                sbs_list.append(i - ind)

            b -= 1
            if b == -1:
                b = batch_size - 1

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)

        i += 1
    while len(missed) > 0:
            sbs_list.append(i - missed.pop())
    
    return sbs_list
    

In [8]:
def start_result_collection(filename):
    
    global project_ytest_lib
    
    results = pd.read_csv(filename)    
    projects = set(results['project'].tolist())
    all_versions = pd.DataFrame()
        
    for p in projects:
        
        #getting project data
        p_data = results[ results['project']==p]
        pframe = pd.DataFrame()
        
        #splitting data into versions
        versions = separate_versions(p_data)
        
        for start in range(0,10):
            if len(versions[start]) > 0:
                pframe = versions[start]
                break
        
        #starting with the first version's project frame
        for x in range(len(pframe)):
            row = pframe.iloc[x] 
            alg = row['algorithm']
            b = row['batch_size']
            
            final_ci = str_to_list(row['ci'])
            
            
            #appending other frames to outer frame
            for i in range(start+1,10):
                next_ver = versions[i]
                
                #extracting corresponding outer row for each version
                new_df = next_ver[ (next_ver['algorithm']==alg) & (next_ver['batch_size']==b)]
                
                if len(new_df) > 0:
                    new_row = new_df.iloc[0]
                else:
                    continue
                
                total_reqd_builds = (row['builds_reqd']*row['testall_size']) + (new_row['builds_reqd']*new_row['testall_size'])
                failures_found = (row['failures_found']*row['testall_size']) + (new_row['failures_found']*new_row['testall_size'])
                failures_not_found = (row['failures_not_found']*row['testall_size']) + (new_row['failures_not_found']*new_row['testall_size'])
                total_size = row['testall_size'] + new_row['testall_size']
                
                row['builds_reqd'] = total_reqd_builds/total_size
                row['failures_found'] = failures_found/total_size
                row['failures_not_found'] = failures_not_found/total_size
                row['testall_size'] = total_size
                
                new_ci = str_to_list(new_row['ci'])
                final_ci.extend(new_ci)
                
                row['batch_delays'] += new_row['batch_delays']
                row['sbs_delays'] += new_row['sbs_delays']
                row['bad_builds'] += new_row['bad_builds']
                row['ci'] = final_ci
                                                            
            pframe.iloc[x] = row
        all_versions = all_versions.append(pframe)
    
    return all_versions

In [11]:
filenames = ['trial_aici_results.csv']
file_root = '../'
dfs = []

for file in filenames:
    dfs.append(start_result_collection(file_root+file))

#combining dfs
final_df = pd.DataFrame()
final_df = dfs[0]

for i in range(1, len(dfs)):
    final_df = final_df.append(dfs[i])

final_df.to_csv('combined_results.csv')

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [12]:
final_df = final_df.loc[:, ~final_df.columns.str.contains('^Unnamed')]

In [13]:
final_df.to_csv('combined_results.csv')